In [1]:
!pip install transformers pandas torch
!pip install --upgrade transformers

In [2]:
!pip install transformers[torch] bitsandbytes accelerate
!pip install bitsandbytes





In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd

In [4]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-fvh88gry
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-fvh88gry
  Resolved https://github.com/huggingface/transformers.git to commit baa3b22137d9d47097bd5a17736c0639ecf38e5b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [25]:
!pip install accelerate

In [5]:
#Use this model-allenai/OLMo-2-1124-13B-instruct

# Step 1: Load the pre-trained OLMo model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-2-1124-13B-SFT", trust_remote_code=True)
# Load the model using 4-bit quantization for memory efficiency
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_use_double_quant=True,  # Use double quantization
)
model = AutoModelForCausalLM.from_pretrained(
    "allenai/OLMo-2-1124-13B-SFT",
    quantization_config=quantization_config,
    device_map="auto",  # Automatically place layers on available devices
    trust_remote_code=True  # Trust the remote code from the model repository
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/37.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00006.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

pytorch_model-00003-of-00006.bin:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

pytorch_model-00004-of-00006.bin:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

pytorch_model-00005-of-00006.bin:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/2.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

In [20]:
# Step 2: Function to split text into sentences
def split_sentences(text):
    sentences = text.split(". ")
    sentences = [sentence.strip() + "." if not sentence.endswith('.') else sentence.strip() for sentence in sentences]
    return sentences

In [7]:
# Step 3: Function to determine the relationship between two sentences
def get_relationship(sentence1, sentence2):
    prompt = f"""
    Determine the relationship between the following sentences:
    Sentence 1: "{sentence1}"
    Sentence 2: "{sentence2}"
    Possible relationships: [Causal, Conditional, Sequential, Comparison, Contradiction]
    Output: The relationship is
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=50, padding="max_length")

    # Move input_ids to the correct device (e.g., CUDA)
    inputs['input_ids'] = inputs['input_ids'].to(model.device)
    inputs['attention_mask'] = inputs['attention_mask'].to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=50)
    relationship = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return relationship.split(":")[-1].strip()  # Extract the relationship

In [8]:
# Step 4: Function to generate CSV of sentence pairs and their relationships from the uploaded CSV file
def generate_relationship_csv_from_uploaded_file(input_csv, output_csv="sentence_relationships.csv"):
    # Read the input CSV file
    df = pd.read_csv(input_csv)

    # Assuming the CSV file has a column "text" with the content to process
    relationships = []  # List to store relationships

    for index, row in df.iterrows():
        text = row['Content']  # Get the text from the "text" column

        sentences = split_sentences(text)  # Split the text into sentences

        # Iterate over consecutive sentence pairs (1st with 2nd, 2nd with 3rd, etc.)
        for i in range(len(sentences) - 1):  # Ensure we don't go out of bounds
            sentence1 = sentences[i]
            sentence2 = sentences[i + 1]

            # Get the relationship between the pair of sentences
            relationship = get_relationship(sentence1, sentence2)

            # Store the pair of sentences and their relationship
            relationships.append({
                "Sentence 1": sentence1,
                "Sentence 2": sentence2,
                "Relationship": relationship
            })

    # Create a pandas DataFrame from the relationships list
    df_relationships = pd.DataFrame(relationships)

    # Save the DataFrame to a CSV file
    df_relationships.to_csv(output_csv, index=False)
    print(f"CSV file with sentence relationships saved to {output_csv}")

# Example usage:
# Assuming you have uploaded a file called "input_texts.csv" that contains a column "text" with paragraphs
# The output CSV will be saved as "sentence_relationships.csv"

input_csv = "/content/summarized_Abdomen.csv"  # Replace with the path to your uploaded CSV file
generate_relationship_csv_from_uploaded_file(input_csv, "sentence_relationships.csv")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


CSV file with sentence relationships saved to sentence_relationships.csv
